# **Overview**

# **Library and Configurations**

In [ ]:
import sys
sys.path.append('..')

from warnings import filterwarnings
filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [ ]:
import os
import glob
import cv2
import re
import math
import warnings

warnings.filterwarnings('ignore')

# Path
base_train_path = '../data/raw/Train'
output_path = '../data/extracted'

# Cleaning Column
def clean_column_names(df):
    """
    Membersihkan dan menyeragamkan nama kolom pada DataFrame.
    """
    new_columns = []
    for col in df.columns:
        cleaned_col = col.strip().replace(' ', '_').lower()
        cleaned_col = re.sub(r'[().%°/]', '', cleaned_col)
        new_columns.append(cleaned_col)
    df.columns = new_columns
    return df

# Plot Extraction
def find_top_border(gray_image, plot_x_start, plot_x_end):
    """
    Menemukan posisi piksel Y dari garis batas atas plot
    dengan metode horizontal projection.
    """
    plot_area = gray_image[:, plot_x_start:plot_x_end]
    _, binary_inv = cv2.threshold(plot_area, 200, 255, cv2.THRESH_BINARY_INV)

    horizontal_projection = np.sum(binary_inv, axis=1)
    for y, pixel_sum in enumerate(horizontal_projection[10:], start=10):
        if pixel_sum > (plot_area.shape[1] * 0.20 * 255):
            return y

    return None

def label_extraction(image_path):
    """
    Mengekstrak label Y, memprediksi nilai 0, dan juga memprediksi
    nilai maksimum absolut dari plot berdasarkan garis batas atas.
    """
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Gambar tidak ditemukan di {image_path}")
            return None, None

        output_image = image.copy()
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # --- Bagian Ekstraksi Label OCR (Tetap Sama) ---
        y_start, y_end = 0, 617
        x_start, x_end = 0, 1500
        y_axis_roi = gray[y_start:y_end, x_start:x_end]
        y_axis_roi_clean = cv2.threshold(y_axis_roi, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        config = r'--oem 3 --psm 6 outputbase digits'
        data = pytesseract.image_to_data(y_axis_roi_clean, config=config, output_type=pytesseract.Output.DICT)

        label_lokasi = []

        n_boxes = len(data['level'])
        for i in range(n_boxes):
            text = data['text'][i].strip()
            if text.isdigit() and int(text) < 200:
                (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
                real_x = x + x_start + w/2
                real_y = y + y_start + h/2
                if int(text)%5 == 0:
                    label_lokasi.append({
                        'text': int(text), 'x': real_x, 'y': real_y, 'width': w, 'height': h
                    })
                else:
                    continue

        if len(label_lokasi) < 2:
            for i in range(n_boxes):
                text = data['text'][i].strip()
                if text.isdigit() and int(text) < 200:
                    (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
                    real_x = x + x_start + w/2
                    real_y = y + y_start + h/2
                    label_lokasi.append({
                        'text': int(text), 'x': real_x, 'y': real_y, 'width': w, 'height': h
                    })

        sorted_lokasi = sorted(label_lokasi, key=lambda item: item['text'])

        p_max_1 = sorted_lokasi[-1]
        p_max_2 = sorted_lokasi[-2]

        pixel_diff = abs(p_max_2['y'] - p_max_1['y'])
        value_diff = abs(p_max_2['text'] - p_max_1['text'])

        avg_pixel_per_value = pixel_diff / value_diff

        # Prediksi label 0
        ref_label_min = sorted_lokasi[-1]
        pixel_step_to_zero = ref_label_min['text'] * avg_pixel_per_value
        predicted_y_zero = ref_label_min['y'] + pixel_step_to_zero
        label_lokasi.append({
            'text': 0, 'x': ref_label_min['x'], 'y': predicted_y_zero, 'width': ref_label_min['width'], 'height': ref_label_min['height']
        })

        # Prediksi Nilai Maksimum Berdasarkan Border
        data_boundaries = find_data_boundaries(image_path)
        plot_x_start, plot_x_end_border = data_boundaries['data_start'], data_boundaries['data_end']
        top_border_y = find_top_border(gray, plot_x_start, plot_x_end_border)

        if top_border_y:
            ref_label_max = sorted_lokasi[-1]
            pixel_dist_to_border = ref_label_max['y'] - top_border_y
            value_increase = pixel_dist_to_border / avg_pixel_per_value
            predicted_max_val = ref_label_max['text'] + value_increase
            label_lokasi.append({
                'text': predicted_max_val, 'x': ref_label_max['x'], 'y': top_border_y, 'width': ref_label_max['width'], 'height': ref_label_max['height']
            })

        final_dict = {item['text']: (item['x'], item['y']) for item in sorted(label_lokasi, key=lambda item: item['text'])}
        return final_dict

    except Exception as e:
        print(f"Terjadi error: {e}")
        return None

def find_data_boundaries(image_path: str) -> dict:
    """
    Mencari titik mulai dan selesainya data dalam satu plot
    """
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Gambar tidak ditemukan di {image_path}")

        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        lower_blue = np.array([100, 50, 30])
        upper_blue = np.array([130, 255, 255])
        color_mask = cv2.inRange(hsv, lower_blue, upper_blue)

        contours, _ = cv2.findContours(color_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if not contours:
            raise ValueError("Tidak ada objek biru yang terdeteksi pada gambar.")

        all_x_coords = []
        for cnt in contours:
            x_coords_in_contour = cnt[:, 0, 0]
            all_x_coords.extend(x_coords_in_contour)

        if not all_x_coords:
            raise ValueError("Gagal mengekstrak koordinat dari objek biru.")

        data_start = min(all_x_coords)
        data_end = max(all_x_coords)

        return {"data_start": data_start, "data_end": data_end}

    except Exception as e:
        print(f"Terjadi error: {e}")
        return None

def extract_rainfall_from_plot(image_path, total_days):
    """
    Mengekstrak informasi berupa titik titik pada plot kemudian
    dikonversi menjadi informasi data berdasarkan jumlah pixel
    """
    try:
        original_image = cv2.imread(image_path)
        if original_image is None:
            raise FileNotFoundError(f"Gambar tidak ditemukan di {image_path}")

        hsv = cv2.cvtColor(original_image, cv2.COLOR_BGR2HSV)
        gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
        lower_blue = np.array([100, 50, 30])
        upper_blue = np.array([130, 255, 255])
        blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)

        opening_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        mask_without_lines = cv2.morphologyEx(blue_mask, cv2.MORPH_OPEN, opening_kernel)

        erosion_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1,1))
        mask_with_merged_dots = cv2.erode(mask_without_lines, erosion_kernel, iterations=2)

        sure_bg = cv2.dilate(mask_with_merged_dots, opening_kernel, iterations=3)

        dist_transform = cv2.distanceTransform(mask_with_merged_dots, cv2.DIST_L2, 5)
        _, sure_fg = cv2.threshold(dist_transform, 0.1 * dist_transform.max(), 255, 0)
        sure_fg = np.uint8(sure_fg)

        unknown = cv2.subtract(sure_bg, sure_fg)
        _, markers = cv2.connectedComponents(sure_fg)
        markers = markers + 1
        markers[unknown == 255] = 0

        img_for_watershed = cv2.bitwise_and(original_image, original_image, mask=mask_with_merged_dots)
        markers = cv2.watershed(img_for_watershed, markers)

        detected_points = []
        for label in np.unique(markers):
            if label <= 1:
                continue

            single_dot_mask = np.zeros(gray.shape, dtype="uint8")
            single_dot_mask[markers == label] = 255
            contours, _ = cv2.findContours(single_dot_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if contours:
                M = cv2.moments(contours[0])
                if M["m00"] != 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                    detected_points.append((cX, cY))

        labels = label_extraction(image_path)
        min_y_val = min(labels.keys())
        max_y_val = max(labels.keys())
        min_y_pixel = labels[min_y_val][1]
        max_y_pixel = labels[max_y_val][1]

        detected_points.sort(key=lambda p: p[0])

        data_coordinates = find_data_boundaries(image_path)
        plot_x_start = data_coordinates['data_start']
        plot_x_end = data_coordinates['data_end']

        plot_width = plot_x_end - plot_x_start

        daily_rainfall_values = [0.0] * total_days

        pixel_span_y = min_y_pixel - max_y_pixel
        value_span = max_y_val - min_y_val

        for x_pixel, y_pixel in detected_points:
            day_index = int(((x_pixel - plot_x_start) / plot_width) * total_days)
            if 0 <= day_index < total_days:
                if pixel_span_y > 0:
                    rainfall_value = max_y_val - ((y_pixel - max_y_pixel) / pixel_span_y) * value_span
                    current_value = daily_rainfall_values[day_index]
                    daily_rainfall_values[day_index] = max(current_value, rainfall_value)

        daily_rainfall_values = [max(0, val) for val in daily_rainfall_values]

    except error as e:
        Print(f'Terjadi eror: {e}')

    return daily_rainfall_values

# Merge Extracted Data
def proses_dan_gabungkan_plot_dengan_tanggal(path_induk, path_keluaran):
    print("Memulai proses penggabungan...")
    os.makedirs(path_keluaran, exist_ok=True)
    print(f"Folder output dipastikan ada di: {path_keluaran}")
    folder_lokasi = [f.name for f in os.scandir(path_induk) if f.is_dir()]
    print(f"Ditemukan {len(folder_lokasi)} folder lokasi untuk diproses.")

    for nama_lokasi in folder_lokasi:
        print(f"\n-> Memproses: '{nama_lokasi}'...")
        path_lokasi_spesifik = os.path.join(path_induk, nama_lokasi)
        list_df_per_tahun = []
        csv_files = sorted(glob.glob(os.path.join(path_lokasi_spesifik, "*.csv")))
        png_files = sorted(glob.glob(os.path.join(path_lokasi_spesifik, "*.png")))

        for csv_file, png_file in zip(csv_files, png_files):
            df_tahun = pd.read_csv(csv_file)
            df_tahun = clean_column_names(df_tahun)
            df_tahun['date'] = pd.to_datetime(df_tahun['date'])
            num_days = len(df_tahun)

            print(f"  -- Mengekstrak {os.path.basename(png_file)} untuk {num_days} hari...")
            rainfall_values = extract_rainfall_from_plot(png_file, num_days)

            if len(rainfall_values) == num_days:
                df_tahun['daily_rainfall_total_mm'] = rainfall_values
                list_df_per_tahun.append(df_tahun)
            else:
                print(f"  ❌ GAGAL sinkronisasi untuk {os.path.basename(csv_file)}. Diharapkan {num_days}, didapat {len(rainfall_values)}.")

        if list_df_per_tahun:
            df_lokasi_final = pd.concat(list_df_per_tahun, ignore_index=True)
            df_hasil = df_lokasi_final[['date', 'daily_rainfall_total_mm']]

            nama_file_output = f"{nama_lokasi}_target_extracted_FINAL.csv"
            path_file_output = os.path.join(path_keluaran, nama_file_output)
            df_hasil.to_csv(path_file_output, index=False)
            print(f"  ✅ Berhasil! {len(df_hasil)} baris data disimpan untuk '{nama_lokasi}'.")
        else:
            print(f"  -- Tidak ada data yang berhasil diproses untuk '{nama_lokasi}'.")

    print("\n\n🎉 Semua proses selesai!")

# Running
proses_dan_gabungkan_plot_dengan_tanggal(base_train_path, output_path)

In [ ]:
import pandas as pd
import os
import glob

# --- PENGATURAN PATH ---
# Path ke folder berisi fitur yang sudah bersih (9 kolom)
path_fitur = '../data/merged/'

# Path ke folder berisi target yang sudah diekstrak (2 kolom: date, daily_rainfall_total_mm)
path_target = '../data/extracted/'

# Path dan nama file untuk output final
output_file = '../data/processed/train.csv'

# --- LOGIKA UTAMA ---
def train_dataset(path_data_fitur, path_data_target, path_output):
    """
    Menggabungkan file fitur dan target per lokasi, lalu menyatukan semuanya.
    """
    try:
        daftar_file_fitur = glob.glob(os.path.join(path_data_fitur, "*.csv"))
        # Contoh: dari 'Admiralty_gabungan_bersih.csv' menjadi 'Admiralty'
        nama_lokasi_list = [os.path.basename(f).replace('_merged.csv', '') for f in daftar_file_fitur]
        print(f"Ditemukan {len(nama_lokasi_list)} lokasi untuk diproses.")
    except FileNotFoundError:
        print(f"❌ Error: Folder fitur tidak ditemukan di '{path_data_fitur}'.")
        return

    all_locations_data = []

    # Loop melalui setiap nama lokasi
    for nama_lokasi in nama_lokasi_list:
        print(f"\n-> Memproses: '{nama_lokasi}'...")

        file_fitur_path = os.path.join(path_data_fitur, f"{nama_lokasi}_merged.csv")
        file_target_path = os.path.join(path_data_target, f"{nama_lokasi}_target_extracted_FINAL.csv")

        if not os.path.exists(file_fitur_path) or not os.path.exists(file_target_path):
            print(f"  -- Lewati '{nama_lokasi}', salah satu atau kedua file tidak ditemukan.")
            continue

        df_fitur = pd.read_csv(file_fitur_path)
        df_target = pd.read_csv(file_target_path)

        df_fitur['date'] = pd.to_datetime(df_fitur['date'])
        df_target['date'] = pd.to_datetime(df_target['date'])

        df_lengkap_lokasi = pd.merge(df_fitur, df_target, on='date', how='inner')
        df_lengkap_lokasi['lokasi'] = nama_lokasi

        print(f"  -- Penggabungan berhasil, menghasilkan {len(df_lengkap_lokasi)} baris data.")
        all_locations_data.append(df_lengkap_lokasi)

    if all_locations_data:
        final_dataset = pd.concat(all_locations_data, ignore_index=True)
        final_dataset.to_csv(path_output, index=False)

        print("\n\n🎉 Dataset training final berhasil dibuat!")
        print(f"Disimpan di: {path_output}")
        print("\n--- Informasi Dataset Final ---")
        final_dataset.info()
    else:
        print("\nTidak ada data yang berhasil diproses.")

# --- JALANKAN PROSES ---
train_dataset(path_fitur, path_target, output_file)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

# Path
input_file = '../data/processed/train.csv'

# Visualize
def tampilkan_plot_satu_lokasi(df, nama_lokasi):
    """
    Membuat dan menampilkan plot time series curah hujan harian
    untuk satu lokasi yang dipilih.
    """

    print(f"Mencari data untuk lokasi: '{nama_lokasi}'...")

    df_lokasi = df[df['lokasi'] == nama_lokasi].copy()

    if df_lokasi.empty:
        print(f"❌ Error: Lokasi '{nama_lokasi}' tidak ditemukan di dalam dataset.")
        print(f"Lokasi yang tersedia: {sorted(df['lokasi'].unique())}")
        return

    print(f"-> Data ditemukan. Membuat plot untuk '{nama_lokasi}'...")

    df_lokasi.sort_values('date', inplace=True)

    sns.set_style("darkgrid")

    plt.figure(figsize=(18, 7))

    plt.plot(df_lokasi['date'], df_lokasi['daily_rainfall_total_mm'], color='dodgerblue', linewidth=1)
    plt.title(f'Curah Hujan Harian: {nama_lokasi}', fontsize=18, fontweight='bold')
    plt.xlabel('Tanggal', fontsize=12)
    plt.ylabel('Total Curah Hujan Harian (mm)', fontsize=12)
    plt.ylim(bottom=-10)

    plt.tight_layout()
    plt.show()

# Running
try:
    df_lengkap = pd.read_csv(input_file)
    df_lengkap['date'] = pd.to_datetime(df_lengkap['date'])


    lokasi_yang_dipilih = 'Bukit_Panjang'


    tampilkan_plot_satu_lokasi(df_lengkap, lokasi_yang_dipilih)

except FileNotFoundError:
    print(f"❌ Error: File dataset tidak ditemukan di '{input_file}'. Pastikan path sudah benar.")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

# --- FUNGSI BARU UNTUK MENAMPILKAN PLOT ---
def tampilkan_plot_per_lokasi_dan_tahun(df, nama_lokasi, tahun):
    """
    Membuat dan menampilkan plot time series curah hujan harian
    untuk satu lokasi dan satu tahun yang dipilih.
    """
    print(f"Mencari data untuk lokasi: '{nama_lokasi}' pada tahun: {tahun}...")

    # 1. Saring data untuk lokasi yang dipilih
    df_lokasi = df[df['lokasi'] == nama_lokasi]

    # Cek apakah lokasi ditemukan
    if df_lokasi.empty:
        print(f"❌ Error: Lokasi '{nama_lokasi}' tidak ditemukan di dalam dataset.")
        return

    # 2. Saring lebih lanjut untuk tahun yang dipilih
    # Pastikan kolom 'date' sudah dalam format datetime
    df_plot = df_lokasi[df_lokasi['date'].dt.year == tahun].copy()

    # Cek apakah data untuk tahun tersebut ada
    if df_plot.empty:
        print(f"❌ Error: Tidak ada data untuk tahun {tahun} di lokasi '{nama_lokasi}'.")
        return

    print(f"-> Data ditemukan. Membuat plot...")

    # Urutkan data berdasarkan tanggal
    df_plot.sort_values('date', inplace=True)

    # 3. Membuat plot
    sns.set_style("darkgrid")
    plt.figure(figsize=(18, 7))

    plt.plot(df_plot['date'], df_plot['daily_rainfall_total_mm'], color='dodgerblue', marker='o', linestyle='-', markersize=4, alpha=0.7)

    # 4. Kustomisasi plot
    plt.title(f'Curah Hujan Harian: {nama_lokasi} - Tahun {tahun}', fontsize=18, fontweight='bold')
    plt.xlabel('Tanggal', fontsize=12)
    plt.ylabel('Total Curah Hujan Harian (mm)', fontsize=12)
    plt.ylim(bottom=-10)

    # Mengatur format tanggal di sumbu-x agar lebih rapi
    plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%b %d'))
    plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.MonthLocator())
    plt.gcf().autofmt_xdate() # Memiringkan tanggal agar tidak tumpang tindih

    plt.tight_layout()

    # 5. Tampilkan plot
    plt.show()


# --- JALANKAN PROSES ---

# 1. Baca dataset
try:
    df_lengkap = pd.read_csv(input_file)
    df_lengkap['date'] = pd.to_datetime(df_lengkap['date'])

    # ==========================================================
    # ===> UBAH NAMA LOKASI DAN TAHUN DI SINI UNTUK MENCOBA <===
    lokasi_yang_dipilih = 'Bukit_Panjang'
    tahun_yang_dipilih  = 2020
    # ==========================================================

    # 2. Panggil fungsi untuk membuat dan menampilkan plot
    tampilkan_plot_per_lokasi_dan_tahun(df_lengkap, lokasi_yang_dipilih, tahun_yang_dipilih)

except FileNotFoundError:
    print(f"❌ Error: File dataset tidak ditemukan di '{input_file}'. Pastikan path sudah benar.")

In [ ]:
folder_total = '../data/processed/train.csv'
lengkap = pd.read_csv(folder_total)
lengkap.head()

In [ ]:
lengkap.info()

In [ ]:
# Path
file_path = input_file
output_file_path = '../data/clean/train.csv'

try:
    df = pd.read_csv(file_path)
    print("--- Tipe Data SEBELUM Perubahan ---")
    df.info()

    kolom_untuk_diubah = [
        'highest_30_min_rainfall_mm',
        'highest_60_min_rainfall_mm',
        'highest_120_min_rainfall_mm',
        'mean_temperature_c',
        'maximum_temperature_c',
        'minimum_temperature_c',
        'mean_wind_speed_km/h',
        'max_wind_speed_km/h'
    ]

    print("\nMemulai proses konversi...")
    for col in kolom_untuk_diubah:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            print(f"  -> Kolom '{col}' berhasil diubah ke tipe numerik.")

    df.rename(columns = {'mean_wind_speed_km/h': 'mean_wind_speed_kmh',
                        'max_wind_speed_km/h' : 'max_wind_speed_kmh'},
              inplace = True)

    print("\n--- Tipe Data SETELAH Perubahan ---")
    df.info()

    print(f"\nMenyimpan DataFrame yang sudah dibersihkan ke file baru...")
    df.to_csv(output_file_path, index=False)
    print(f"✅ Berhasil! File baru telah disimpan di: '{output_file_path}'")

except FileNotFoundError:
    print(f"❌ Error: File tidak ditemukan di '{file_path}'. Pastikan path sudah benar.")